# PROJECT. Исследование поведения пользователей

### ЗАДАНИЕ

Необходимо проверить:

1. Есть ли зависимость между выбранным уровнем сложности и вероятностью оплаты;

2. Различается ли временной промежуток между регистрацией и оплатой у групп пользователей с разным уровнем сложности.

#### ПЛАН

1. Получите данные из файла.
2. Произведите обзор данных и преобразование данных, если оно необходимо.
3. Выделите группы пользователей (по аналогии с группами, которые мы выделяли, когда находили зависимость между фактом прохождения обучения и оплатой).
4. Рассчитайте для каждой группы процент оплат (это мы тоже уже делали).
5. Для каждой группы сделайте по два датафрейма: один — с событиями выбора уровня сложности, другой — с событиями оплаты.
6. Объедините датафреймы в рамках одной группы и найдите разницу во времени между событиями (по аналогии с расчётом, который мы уже производили, когда находили время между событиями для всех пользователей).
7. Рассчитайте среднее время между событиями.

In [1]:
import pandas as pd

### Таблица Event
Хранит данные о событиях, которые совершают пользователи. По сути, каждое событие — это факт прохождения пользователем какого-либо этапа игры.

* id	идентификатор события
* user_id	уникальный идентификатор пользователя, совершившего событие в приложении
* start_time	дата и время события
* event_type	тип события, значения: 
   * registration — регистрация; 
   * tutorial_start — начало обучения; 
   * tutorial_finish — завершение обучения; 
   * level_choice — выбор уровня сложности; 
   * pack_choice — выбор пакетов вопросов
* tutorial_id	идентификатор обучения (этот идентификатор есть только у событий обучения)
* selected_level	выбранный уровень сложности обучения


In [2]:
event = pd.read_csv('data/7_4_Events.csv')
event.head()

,id,event_type,selected_level,start_time,tutorial_id,user_id
0,28903,registration,NaN,2016-05-11T23:40:55,NaN,12583
1,28904,registration,NaN,2016-05-11T23:49:58,NaN,12584
2,28905,registration,NaN,2016-05-12T00:53:07,NaN,12585
3,28906,tutorial_start,NaN,2016-05-12T01:32:20,17562.0,12585
4,28907,tutorial_finish,NaN,2016-05-12T01:34:53,17562.0,12585


In [3]:
"""выбираем только тех пользователей, которые зарегистрировались в 2018 г"""

mask = (event['start_time']>='2018-01-01') & (event['start_time']<'2019-01-01') & (event['event_type']=='registration')
registered = event[mask]['user_id'].to_list()
events_df = event[event['user_id'].isin(registered)]
events_df['start_time'] = pd.to_datetime(events_df['start_time'])
events_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 66959 entries, 51405 to 118364
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   id              66959 non-null  int64         
 1   event_type      66959 non-null  object        
 2   selected_level  8342 non-null   object        
 3   start_time      66959 non-null  datetime64[ns]
 4   tutorial_id     32954 non-null  float64       
 5   user_id         66959 non-null  int64         
dtypes: datetime64[ns](1), float64(1), int64(2), object(2)
memory usage: 3.6+ MB


<ipython-input-3-5db9283e6810>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  events_df['start_time'] = pd.to_datetime(events_df['start_time'])


In [13]:
""" Оценим, какое количество пользователей совершали события:"""
events_df['user_id'].nunique()

19926

In [5]:
"""Изучаем данные, по которым были выбраны уровни сложности"""
events_df['selected_level'].value_counts()

medium    4645
easy      2448
hard      1249
Name: selected_level, dtype: int64

#### Исследуем пропуски

In [9]:
"""посмотрим на events_df, оставив в нём только такие строки, где selected_level = hard"""

events_df[events_df['selected_level'] == 'hard'].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1249 entries, 51428 to 118300
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   id              1249 non-null   int64         
 1   event_type      1249 non-null   object        
 2   selected_level  1249 non-null   object        
 3   start_time      1249 non-null   datetime64[ns]
 4   tutorial_id     0 non-null      float64       
 5   user_id         1249 non-null   int64         
dtypes: datetime64[ns](1), float64(1), int64(2), object(2)
memory usage: 68.3+ KB


In [10]:
"""посмотрим на events_df, оставив в нём только такие строки, где selected_level = medium"""

events_df[events_df['selected_level'] == 'medium'].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4645 entries, 51424 to 118363
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   id              4645 non-null   int64         
 1   event_type      4645 non-null   object        
 2   selected_level  4645 non-null   object        
 3   start_time      4645 non-null   datetime64[ns]
 4   tutorial_id     0 non-null      float64       
 5   user_id         4645 non-null   int64         
dtypes: datetime64[ns](1), float64(1), int64(2), object(2)
memory usage: 254.0+ KB


In [11]:
"""посмотрим на events_df, оставив в нём только такие строки, где selected_level = easy"""

events_df[events_df['selected_level'] == 'easy'].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2448 entries, 51450 to 118362
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   id              2448 non-null   int64         
 1   event_type      2448 non-null   object        
 2   selected_level  2448 non-null   object        
 3   start_time      2448 non-null   datetime64[ns]
 4   tutorial_id     0 non-null      float64       
 5   user_id         2448 non-null   int64         
dtypes: datetime64[ns](1), float64(1), int64(2), object(2)
memory usage: 133.9+ KB


#### Исследуем значения

In [12]:
"""Вызовем метод describe(), чтобы оценить характеристики каждого столбца. 
По умолчанию метод describe() выдаёт характеристики только по столбцам с численными 
типами (например, int64, float64). Мы вызовем этот метод с параметром include='all'
для того, чтобы отображать характеристики для всех столбцов."""

events_df.describe(include='all')

<ipython-input-12-f62b66a4a67c>:6: FutureWarning: Treating datetime data as categorical rather than numeric in `.describe` is deprecated and will be removed in a future version of pandas. Specify `datetime_is_numeric=True` to silence this warning and adopt the future behavior now.
  events_df.describe(include='all')


,id,event_type,selected_level,start_time,tutorial_id,user_id
count,66959.000000,66959,8342,66959,32954.000000,66959.000000
unique,NaN,5,3,66809,NaN,NaN
top,NaN,registration,medium,2018-03-13 09:37:43,NaN,NaN
freq,NaN,19926,4645,9,NaN,NaN
first,NaN,NaN,NaN,2018-01-01 03:48:40,NaN,NaN
last,NaN,NaN,NaN,2019-01-01 05:50:36,NaN,NaN
mean,113787.000045,NaN,NaN,NaN,40532.934393,37781.543362
std,19329.542752,NaN,NaN,NaN,5213.486632,5751.497904
min,80308.000000,NaN,NaN,NaN,31505.000000,27832.000000
25%,97047.500000,NaN,NaN,NaN,36008.250000,32849.000000


### Таблица purchase
Хранит данные об оплатах, которые совершают пользователи.

id	идентификатор события

user_id	уникальный идентификатор пользователя, совершившего событие в приложении

event_datetime	дата и время события/покупки

amount	сумма оплаты

In [14]:
purchase = pd.read_csv('data/purchase.csv')
display(purchase.head())
display(purchase.info())

,id,user_id,event_datetime,amount
0,15674,12584,2016-05-12T10:34:16,100
1,15675,12985,2016-05-13T08:25:56,50
2,15676,12828,2016-05-13T16:33:46,50
3,15677,12598,2016-05-14T01:09:37,150
4,15678,13037,2016-05-14T01:24:46,100


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5956 entries, 0 to 5955
Data columns (total 4 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   id              5956 non-null   int64 
 1   user_id         5956 non-null   int64 
 2   event_datetime  5956 non-null   object
 3   amount          5956 non-null   int64 
dtypes: int64(3), object(1)
memory usage: 186.2+ KB


None

In [ ]:
"""Также у нас есть одинаковые столбцы id в двух датафреймах, но смысл их 
несколько отличается, так как столбец id в events_df указывает на идентификатор 
события, а столбец id в purchase_df указывает на идентификатор оплаты. Поэтому 
применим функцию rename(), чтобы переименовать столбцы в датафреймах."""

events_df = events_df.rename(columns={"id": "event_id"})
purchase_df = purchase_df.rename(columns={"id": "purchase_id"})

In [15]:
""" Фильтруем только тех, кто зарегистрировался в 2018г"""

purchase_df = purchase[purchase['user_id'].isin(registered)]
purchase_df['event_datetime'] = pd.to_datetime(purchase_df['event_datetime'])
purchase_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1600 entries, 1171 to 2778
Data columns (total 4 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   id              1600 non-null   int64         
 1   user_id         1600 non-null   int64         
 2   event_datetime  1600 non-null   datetime64[ns]
 3   amount          1600 non-null   int64         
dtypes: datetime64[ns](1), int64(3)
memory usage: 62.5 KB


<ipython-input-15-092fe9d89b94>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  purchase_df['event_datetime'] = pd.to_datetime(purchase_df['event_datetime'])


In [16]:
""" Исследуем значения"""

purchase_df.describe()

,id,user_id,amount
count,1600.000000,1600.000000,1600.000000
mean,17645.505625,37752.766250,110.734375
std,462.038637,5822.621784,54.696628
min,16845.000000,27845.000000,25.000000
25%,17245.750000,32815.750000,50.000000
50%,17645.500000,37633.500000,100.000000
75%,18045.250000,43023.000000,150.000000
max,18452.000000,47742.000000,300.000000


In [162]:
""" Для начала посмотрим, сколько пользователей совершают событие: выбирают уровень сложности."""

selected_level_hard_users_count = events_df[events_df['selected_level'] == 'hard']['user_id'].nunique()
selected_level_medium_users_count = events_df[events_df['selected_level'] == 'medium']['user_id'].nunique()
selected_level_easy_users_count = events_df[events_df['selected_level'] == 'easy']['user_id'].nunique()
selected_level_notnull_users_count = events_df[events_df['selected_level'].notnull() == True]['user_id'].nunique()
selected_level_all_users_count = events_df['user_id'].nunique()



In [37]:
selected_level_hard_users_count + selected_level_medium_users_count + selected_level_easy_users_count == selected_level_notnull_users_count

True

In [40]:
percent_selected_level_notnull_users = selected_level_notnull_users_count / selected_level_all_users_count

print(
    "Процент пользователей, выбравшие уровени сложности (от общего числа зарегистрированных пользователей): {:.2%}".format(
        percent_selected_level_notnull_users
    )
)

Процент пользователей, выбравшие уровени сложности (от общего числа зарегистрированных пользователей): 41.86%


#### ВЫВОД
Меньше половины пользователей (41.86 %) доходят до этапа выбора уровня сложности вопросов. А ведь этот этап напрямую влияет на то, что пользователь будет пользоваться приложением через бесплатные возможности, которые в дальнейшем могут привести к оплате.
Таким образом, для успешной монетизации приложения крайне важно оптимизировать прохождение до этапа выбора сложности.

In [38]:
percent_selected_level_hard_users = selected_level_hard_users_count / selected_level_notnull_users_count

print(
    "Процент пользователей, выбравший высокий уровень сложности (от числа пользователей, которые выбирали): {:.2%}".format(
        percent_selected_level_hard_users
    )
)

Процент пользователей, выбравший высокий уровень сложности (от числа пользователей, которые выбирали): 14.97%


In [42]:
percent_selected_level_medium_users = selected_level_medium_users_count / selected_level_notnull_users_count

print(
    "Процент пользователей, выбравший средний уровень сложности (от числа пользователей, которые выбирали): {:.2%}".format(
        percent_selected_level_medium_users
    )
)

Процент пользователей, выбравший средний уровень сложности (от числа пользователей, которые выбирали): 55.68%


In [43]:
percent_selected_level_easy_users = selected_level_easy_users_count / selected_level_notnull_users_count

print(
    "Процент пользователей, выбравший низкий уровень сложности (от числа пользователей, которые выбирали): {:.2%}".format(
        percent_selected_level_easy_users
    )
)

Процент пользователей, выбравший низкий уровень сложности (от числа пользователей, которые выбирали): 29.35%


### Вывод:
Больше половины пользователей выбираю средний уровень срожности, треть низкий и только 15% высокий уровень.

#### Объединяем датафреймы

In [58]:
"""Добавим в датафрейм purchase_df столбец event_type, который будет содержать одно 
значение purchase. Это нужно, чтобы в объединённом датафрейме однозначно выделить 
события оплаты."""

purchase_df['event_type'] = 'purchase'

In [164]:
"""Также у нас есть одинаковые столбцы id в двух датафреймах, но смысл их 
несколько отличается, так как столбец id в events_df указывает на идентификатор 
события, а столбец id в purchase_df указывает на идентификатор оплаты. Поэтому 
применим функцию rename(), чтобы переименовать столбцы в датафреймах."""

events_df = events_df.rename(columns={"id": "event_id"})
purchase_df = purchase_df.rename(columns={"id": "purchase_id"})

In [61]:
"""объединим датафреймы events_df и purchase_df с помощью функции pd.concat() 
и запишем объединённый датафрейм в переменную total_events_df."""

total_events_df = pd.concat([events_df,purchase_df],sort=False)

In [62]:
total_events_df.head()

,event_id,event_type,selected_level,start_time,tutorial_id,user_id,purchase_id,event_datetime,amount
51405,80308.0,registration,NaN,2018-01-01 03:48:40,NaN,27832,NaN,NaT,NaN
51406,80309.0,registration,NaN,2018-01-01 04:07:25,NaN,27833,NaN,NaT,NaN
51407,80310.0,registration,NaN,2018-01-01 08:35:10,NaN,27834,NaN,NaT,NaN
51408,80311.0,registration,NaN,2018-01-01 11:54:47,NaN,27835,NaN,NaT,NaN
51409,80312.0,registration,NaN,2018-01-01 13:28:07,NaN,27836,NaN,NaT,NaN


In [63]:
total_events_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 68559 entries, 51405 to 2778
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   event_id        66959 non-null  float64       
 1   event_type      68559 non-null  object        
 2   selected_level  8342 non-null   object        
 3   start_time      66959 non-null  datetime64[ns]
 4   tutorial_id     32954 non-null  float64       
 5   user_id         68559 non-null  int64         
 6   purchase_id     1600 non-null   float64       
 7   event_datetime  1600 non-null   datetime64[ns]
 8   amount          1600 non-null   float64       
dtypes: datetime64[ns](2), float64(4), int64(1), object(2)
memory usage: 5.2+ MB


In [64]:
""" Сбросим индексы объединённого датафрейма (так как после объединения они 
дублировались и несут мало смысла) с помощью метода reset_index() и отсортируем 
все события по возрастанию времени с помощью sort_values()."""

total_events_df = total_events_df.reset_index(drop=True).sort_values('start_time')

In [65]:
""" Выделим отдельный датафрейм registration_df, который будет содержать только 
события с event_type = registration. Этот датафрейм будет вспомогательным для определения 
времени между регистрацией и выбором уровня сложности."""

registration_df = total_events_df[total_events_df['event_type'] == 'registration']

In [66]:
""" Для определения среднего количества событий на пользователя мы можем воспользоваться 
методом value_counts(), результатом которого будет серия с количеством событий на каждого 
пользователя. Затем, усреднив эти значения между собой, мы получим среднее число событий 
типа registration на одного пользователя."""

registration_df['user_id'].value_counts().mean()

1.0

In [67]:
""" Оставим в датафрейме registration_df только те данные, которые нужны для наших вычислений — 
столбец user_id с идентификатором пользователя и столбец start_time со временем регистрации. 
Также переименуем столбец start_time в столбец registration_time для понятности."""

registration_df = registration_df[["user_id", "start_time"]].rename(
    columns={"start_time": "registration_time"}
)

In [79]:
""" Выделим отдельный датафрейм level_choice_df, который будет содержать только события
с event_type = level_choice (выбор уровня сложности)."""

level_choice_df = total_events_df[total_events_df['event_type'] == 'level_choice']

In [80]:
level_choice_df['user_id'].value_counts().mean()

1.0

In [81]:
"""Так же, как и в случае с датафреймом registration_df, оставим только такие столбцы, 
которые пригодятся нам в дальнейшем. Это столбцы user_id, selected_level, start_time. 
Также переименуем колонку start_time в level_choice_time:"""

level_choice_df = level_choice_df[["user_id", "selected_level", "start_time"]
].rename(columns={"start_time": "level_choice_time"})

In [82]:
print(level_choice_df)

       user_id selected_level   level_choice_time
19       27835         medium 2018-01-01 20:37:22
23       27839           hard 2018-01-01 22:37:50
36       27840         medium 2018-01-02 05:18:42
38       27845           hard 2018-01-02 06:19:18
45       27842           easy 2018-01-02 08:46:03
...        ...            ...                 ...
66932    47732         medium 2018-12-31 09:59:00
66937    47747         medium 2018-12-31 12:24:40
66948    47753         medium 2018-12-31 22:36:19
66956    47755           easy 2019-01-01 05:04:52
66957    47756         medium 2019-01-01 05:42:11

[8342 rows x 3 columns]


In [166]:
""" Объединяем с помощью функции merge(), объединяя данные по параметру 
user_id. Также нам нужно объединить данные с использованием параметра how='inner', это 
позволит в объединённом датафрейме оставить только такие идентификаторы пользователей, 
которые есть в обоих датафреймах. Вспомните SQL, это очень похоже на INNER JOIN."""

merged_df = registration_df.merge(
    level_choice_df, on="user_id", how="inner"
)
merged_df.head()

,user_id,registration_time,selected_level,level_choice_time
0,27835,2018-01-01 11:54:47,medium,2018-01-01 20:37:22
1,27839,2018-01-01 18:24:01,hard,2018-01-01 22:37:50
2,27840,2018-01-01 20:53:21,medium,2018-01-02 05:18:42
3,27842,2018-01-01 23:18:46,easy,2018-01-02 08:46:03
4,27843,2018-01-02 00:02:28,medium,2018-01-02 14:09:58


In [169]:
"""Сделаем столбец timedelta, в котором посчитаем разницу между выбором уровня сложности 
(level_choice_time) и временем регистрации (registration_time):"""

merged_df["timedelta"] = (
    merged_df["level_choice_time"] - merged_df["registration_time"]
)
merged_df.head()

,user_id,registration_time,selected_level,level_choice_time,timedelta
0,27835,2018-01-01 11:54:47,medium,2018-01-01 20:37:22,0 days 08:42:35
1,27839,2018-01-01 18:24:01,hard,2018-01-01 22:37:50,0 days 04:13:49
2,27840,2018-01-01 20:53:21,medium,2018-01-02 05:18:42,0 days 08:25:21
3,27842,2018-01-01 23:18:46,easy,2018-01-02 08:46:03,0 days 09:27:17
4,27843,2018-01-02 00:02:28,medium,2018-01-02 14:09:58,0 days 14:07:30


In [170]:
"""Давайте воспользуемся методом describe(), чтобы понять некоторые важные параметры."""

merged_df['timedelta'].describe()

count                         8342
mean     0 days 07:10:19.169863342
std      0 days 04:33:51.164488800
min                0 days 00:08:15
25%         0 days 03:53:16.500000
50%                0 days 06:03:28
75%         0 days 09:34:58.500000
max                1 days 18:48:25
Name: timedelta, dtype: object

In [171]:
"""Усреднив с помощью функции mean() значения в столбце timedelta, мы получим среднее время,
которое проходит между регистрацией пользователя и выбором уровня сложности:"""

merged_df["timedelta"].mean()

Timedelta('0 days 07:10:19.169863342')

### Вывод:
Среднее время между регистрацией и выбором уровня сложности 7 часов 10 минут

In [86]:
"""Давайте воспользуемся методом describe(), чтобы понять некоторые важные параметры."""

merged_df['timedelta'].describe()

count                         8342
mean     0 days 07:10:19.169863342
std      0 days 04:33:51.164488800
min                0 days 00:08:15
25%         0 days 03:53:16.500000
50%                0 days 06:03:28
75%         0 days 09:34:58.500000
max                1 days 18:48:25
Name: timedelta, dtype: object

#### ВЫВОД:

четверть пользователей тратит меньше 3 часа 53 минут на переход от регистрации к началу обучения;
половина всех пользователей тратит между регистрацией и началом обучения менее 6 часов 03 минут.

In [172]:
hard_merged_df = merged_df[merged_df['selected_level'] == 'hard']
print(hard_merged_df['timedelta'].describe())


count                         1249
mean     0 days 07:18:32.228182546
std      0 days 04:36:04.422579172
min                0 days 00:08:59
25%                0 days 03:58:18
50%                0 days 06:13:02
75%                0 days 10:03:27
max                1 days 06:43:55
Name: timedelta, dtype: object


In [173]:
medium_merged_df = merged_df[merged_df['selected_level'] == 'medium']
print(medium_merged_df['timedelta'].describe())

count                         4645
mean     0 days 07:08:13.360387513
std      0 days 04:31:51.219231339
min                0 days 00:08:15
25%                0 days 03:50:32
50%                0 days 06:01:39
75%                0 days 09:32:00
max                1 days 07:02:41
Name: timedelta, dtype: object


In [174]:
easy_merged_df = merged_df[merged_df['selected_level'] == 'easy']
print(easy_merged_df['timedelta'].describe())

count                         2448
mean     0 days 07:10:06.324754901
std      0 days 04:36:30.973389545
min                0 days 00:09:26
25%         0 days 03:55:05.750000
50%         0 days 06:02:43.500000
75%         0 days 09:23:38.500000
max                1 days 18:48:25
Name: timedelta, dtype: object


Выбор уровня сложности не зависит от времени. С момента регистрации проходит примерно одно и тоже время, вне звисимости от выбранного типа.

In [123]:
purchase_pack_df = total_events_df[total_events_df["event_type"] == "purchase"]
print(purchase_pack_df["user_id"].value_counts().mean())
purchase_pack_df = purchase_pack_df[["user_id", "event_datetime"]].rename(
    columns={"event_datetime": "purchase_time"}
)

1.0


In [130]:
""" Сколько в среднем для всех пользователей проходит времени между событием выбора 
уровня сложности и первой оплатой (purchase)?"""

merged_df_1 = purchase_pack_df.merge(level_choice_df, on="user_id", how="inner")
merged_df_1["timedelta"] = (
merged_df_1["purchase_time"] - merged_df_1["level_choice_time"]
)
print(merged_df_1["timedelta"].mean())
print(merged_df_1["timedelta"].describe())

3 days 17:52:17.719375
count                         1600
mean        3 days 17:52:17.719375
std      2 days 04:37:14.656359486
min                0 days 00:49:20
25%         1 days 21:27:33.750000
50%                3 days 12:59:27
75%         5 days 09:47:33.750000
max               10 days 18:35:09
Name: timedelta, dtype: object


In [124]:
users_with_level_choice = total_events_df[
    total_events_df["event_type"] == "level_choice"
]["user_id"].unique()

In [175]:
""" Сколько в среднем для всех пользователей проходит времени между событием выбора высокого
уровня сложности и первой оплатой (purchase)?"""

hard_merged_df = hard_merged_df[["user_id", "level_choice_time"]]

merged_df_2 = purchase_pack_df.merge(hard_merged_df, on="user_id", how="inner")
merged_df_2["timedelta"] = (
merged_df_2["purchase_time"] - merged_df_2["level_choice_time"]
)
print(merged_df_2["timedelta"].mean())
print(merged_df_2["timedelta"].describe())

3 days 07:20:41.420814479
count                          442
mean     3 days 07:20:41.420814479
std      1 days 21:43:52.953292605
min                0 days 03:26:45
25%         1 days 14:57:23.500000
50%         3 days 03:13:57.500000
75%         4 days 19:16:00.250000
max                8 days 01:18:13
Name: timedelta, dtype: object


In [176]:
""" Сколько в среднем для всех пользователей проходит времени между событием выбора среднего
уровня сложности и первой оплатой (purchase)?"""

medium_merged_df = medium_merged_df[["user_id", "level_choice_time"]]

merged_df_3 = purchase_pack_df.merge(medium_merged_df, on="user_id", how="inner")
merged_df_3["timedelta"] = (
merged_df_3["purchase_time"] - merged_df_3["level_choice_time"]
)
print(merged_df_3["timedelta"].mean())
print(merged_df_3["timedelta"].describe())

3 days 23:14:13.165118679
count                          969
mean     3 days 23:14:13.165118679
std      2 days 06:18:57.618467109
min                0 days 04:18:12
25%                2 days 01:20:07
50%                3 days 19:53:19
75%                5 days 16:07:19
max               10 days 13:51:01
Name: timedelta, dtype: object


In [177]:
""" Сколько в среднем для всех пользователей проходит времени между событием выбора низкого
уровня сложности и первой оплатой (purchase)?"""

easy_merged_df = easy_merged_df[["user_id", "level_choice_time"]]

merged_df_4 = purchase_pack_df.merge(easy_merged_df, on="user_id", how="inner")
merged_df_4["timedelta"] = (
merged_df_4["purchase_time"] - merged_df_4["level_choice_time"]
)
print(merged_df_4["timedelta"].mean())
print(merged_df_4["timedelta"].describe())

3 days 14:58:52.941798941
count                          189
mean     3 days 14:58:52.941798941
std      2 days 07:06:35.644097504
min                0 days 00:49:20
25%                1 days 17:18:56
50%                3 days 06:03:50
75%                5 days 06:58:18
max               10 days 18:35:09
Name: timedelta, dtype: object


В зависимости от выбранного уровня сложности время от принятия решения до оплаты различается незначительно, но все же:
1. При высоком уровне сложности время до оплаты составляет 3 дня 7 часов
2. При среднем уровне сложности время до оплаты составляет 3 дня 23 часа
3. При низком уровне сложности время до оплаты составляет 3 дня 15 часов

In [95]:
""" Сделаем датафрейм purchase_df_1, в котором будут данные по оплатам пользователей, которые 
выбрали уровень сложности."""

purchase_df_1 = purchase_df[purchase_df["user_id"].isin(users_with_level_choice)]

In [96]:
purchase_df_1['user_id'].nunique()

1600

In [105]:
"""Поделив количество пользователей в датафрейме purchase_df_1 на общее количество пользователей, 
которые выбрали уровень сложности, мы получим процент оплативших пользователей в этой группе."""

percent_of_purchase_1 = purchase_df_1["user_id"].nunique() / selected_level_notnull_users_count 
print(
    "Процент пользователей, которые оплатили тренировки (от числа пользователей, выбравших уровень сложности): {:.2%}".format(
        percent_of_purchase_1
    )
)

Процент пользователей, которые оплатили тренировки (от числа пользователей, выбравших уровень сложности): 19.18%


In [113]:
purchase_df_1['amount'].mean()

110.734375

In [108]:

set_selected_level_hard_users_count = set(events_df[events_df['selected_level'] == 'hard']['user_id'].unique())
set_selected_level_medium_users_count = set(events_df[events_df['selected_level'] == 'medium']['user_id'].unique())
set_selected_level_easy_users_count = set(events_df[events_df['selected_level'] == 'easy']['user_id'].unique())
set_selected_level_notnull_users_count = set(events_df[events_df['selected_level'].notnull() == True]['user_id'].unique())

In [110]:
""" Создадим датафрейм purchase_df_2, в котором будут оплаты пользователей, выбравшие 
высокий уровень. И найдём, какой процент таких пользователей оплачивает пакеты вопросов, 
от общего числа пользователей:"""

purchase_df_2 = purchase_df[
    purchase_df["user_id"].isin(set_selected_level_hard_users_count)
]
print(purchase_df_2["user_id"].nunique())
percent_of_purchase_2 = purchase_df_2["user_id"].nunique() / len(
    set_selected_level_hard_users_count
)
print(
    "Процент пользователей, которые оплатили тренировки (от числа пользователей, выбравших высокий (hard) уровень сложности): {:.2%}".format(
        percent_of_purchase_2
    )
)

442
Процент пользователей, которые оплатили тренировки (от числа пользователей, выбравших высокий (hard) уровень сложности): 35.39%


In [114]:
purchase_df_2['amount'].mean()

111.59502262443439

In [111]:
""" Создадим датафрейм purchase_df_3, в котором будут оплаты пользователей, выбравшие 
средний уровень. И найдём, какой процент таких пользователей оплачивает пакеты вопросов, 
от общего числа пользователей:"""

purchase_df_3 = purchase_df[
    purchase_df["user_id"].isin(set_selected_level_medium_users_count)
]
print(purchase_df_3["user_id"].nunique())
percent_of_purchase_3 = purchase_df_3["user_id"].nunique() / len(
    set_selected_level_medium_users_count
)
print(
    "Процент пользователей, которые оплатили тренировки (от числа пользователей, выбравших средний (medium) уровень сложности): {:.2%}".format(
        percent_of_purchase_3
    )
)

969
Процент пользователей, которые оплатили тренировки (от числа пользователей, выбравших средний (medium) уровень сложности): 20.86%


In [115]:
purchase_df_3['amount'].mean()

109.52012383900929

In [112]:
""" Создадим датафрейм purchase_df_4, в котором будут оплаты пользователей, выбравшие 
средний уровень. И найдём, какой процент таких пользователей оплачивает пакеты вопросов, 
от общего числа пользователей:"""

purchase_df_4 = purchase_df[
    purchase_df["user_id"].isin(set_selected_level_easy_users_count)
]
print(purchase_df_4["user_id"].nunique())
percent_of_purchase_4 = purchase_df_4["user_id"].nunique() / len(
    set_selected_level_easy_users_count
)
print(
    "Процент пользователей, которые оплатили тренировки (от числа пользователей, выбравших низкий (easy) уровень сложности): {:.2%}".format(
        percent_of_purchase_4
    )
)

189
Процент пользователей, которые оплатили тренировки (от числа пользователей, выбравших низкий (easy) уровень сложности): 7.72%


In [116]:
purchase_df_4['amount'].mean()

114.94708994708995

### ВЫВОД:
Средняя стоимость оплаты примерно одинакавая для всех 3-х групп и варьируется от 109 до 115 у.е.

Однако доля пользователей, которые совершили покупку после выбора уровня сложности значительно отличаются.

Наиболее перспективной в данном случае группа, которая выбрала высокий уровень сложности: ее конверсия по отношению к группе, которая выбрала низкий уровень выше почти в 5 раз. Возможно данный уровень вопросов более интересен, способен заинтересовать пользователя в продукте. 